<a href="https://colab.research.google.com/github/nath229/My-first-Github-project-with-Eckovation/blob/master/ASTUassignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

To execute the code in the above cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter". To edit the code, just click the cell and start editing.

Variables that you define in one cell can later be used in other cells:

In [55]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import os 
import sys
import matplotlib.pyplot as plt

Foreground Detection: d(x,y,t) = I(x,y,t) - Mean(x,y,t-1)

Change(x,y,t) = d2(x,y,t)<=lmda2 * varience(x,y,t)

which is implementing below

In [56]:
def foreground_change_Detection(img_file,mean,varience,lmda):
    img=cv2.imread(img_file)
    diff=img - mean
    var = varience *(lmda**2) 
    diff_square = np.square(diff)
    I = diff_square - var #Checking the inequality
    mask = np.all(I>0,axis=2)
    return_I = 255*mask.astype(int)
    rerturn_I = return_I.astype(np.uint8)
    return return_I

Initial Background condition:
    Calculating mean and varience

In [57]:
def initial_Background(intial_img):
    arr_img = mpimg.imread(intial_img)
    mean = arr_img
    varience = 9*np.ones(arr_img.shape)
    return mean,varience

The return matrix is divided into some patches and by voting 
decides that for each pixel.(255 or 0)

In [58]:
def voting(return_I,eta,a,b):
    row ,column = return_I.shape
    column_I = np.zeros(( return_I.shape[0], return_I.shape[1]))
    for i in range(a,row - 1 -a):
       for j in range(b,column - 1 -b):
          patch = return_I[i-a:i,j-b:j]
          unique_val, counts = np.unique( patch,return_counts=True)

          if len(counts)==1 and unique_val[0]==1:
              column_I[i,j] = 255
          if len(counts)>1:
              if counts[1]>eta*a*b:
                  column_I[i,j] = 255
    column_I =  column_I.astype(np.uint8)

    return   column_I


Updating the mean and varience images using a weighted average method  

In [59]:
def Update_function(column_I,img_path,mean,varience,alpha):
    img = mpimg.imread(img_path)
    updated_mean = np.zeros(img.shape)
    updated_varience = np.zeros(img.shape)
    diff = img - mean
    diff_square = np.square(diff)
    for  i in range(column_I.shape[0]):
        for  j in range(column_I.shape[1]): 
            if column_I[i,j]==0:#white
                updated_mean[i,j,:] = (1-alpha)*mean[i,j,:] + alpha*img[i,j,:]
                updated_varience[i,j,:] = (1-alpha)*(varience[i,j,:] + alpha*diff_square[i,j,:])
                updated_varience[i,j,:]= np.clip(updated_varience[i,j,:],a_min = 9,a_max=None)
    return updated_mean , updated_varience



In [60]:
from zipfile import ZipFile
zf = ZipFile('/content/AirStripVideoDataSetForBackGroundSubtraction.zip')
zf.extractall()
zf.close()

Main Function starts here
 

In [61]:
def background_substraction(img_directory,num,lmbda,eta,a,b,alpha):
    img_file = os.listdir(img_directory)
    init_image = os.path.join(img_directory,img_file[0])
    mean,varience = initial_Background(init_image)

    for i in range(1,num):
        img_path = os.path.join(img_directory,img_file[i])

        fig ,ax = plt.subplots(1,3,figsize=(15,15))
        return_I = foreground_change_Detection(img_path,mean,varience,lmbda)
        ax[0].imshow(return_I,cmap= 'gray')
        column_I = voting(return_I,eta,a,b)
        mean,varience = Update_function(column_I,img_path,mean,varience,alpha)
        ax[1].imshow(column_I,cmap= 'gray')
        img = cv2.imread(img_path)
        ax[2].imshow(img,cmap= 'gray')
        plt.show()
    return mean,varience

In [62]:
num = int(input("how many time want to scan"))

how many time want to scan10


In [ ]:
mean,varience = background_substraction('/content/AirStripVideo/Images',num,0.7,0.7,7,7,0.8)